### Relacja One to One

Aby zdefiniować relację One to One, tworzymy dwie klasy reprezentujące encję w bazie danych.


In [ ]:
@Entity
@Data
@AllArgsConstructor
@NoArgsConstructor
@Builder
public class Course {

    @Id
    @SequenceGenerator(
            name = "course_sequence",
            sequenceName = "course_sequence",
            allocationSize = 1
    )
    @GeneratedValue(
            strategy = GenerationType.SEQUENCE,
            generator = "course_sequence"
    )

    private Long courseId;
    private String title;
    private Integer credit;
}

####################################################################

@Entity
@Data
@AllArgsConstructor
@NoArgsConstructor
@Builder
public class CourseMaterial {

    @Id
    @SequenceGenerator(
            name = "course_material_sequence",
            sequenceName = "course_material_sequence",
            allocationSize = 1
    )
    @GeneratedValue(
            strategy = GenerationType.SEQUENCE,
            generator = "course_material_sequence"
    )

    private Long courseMaterialId;
    private String url;

    @OneToOne(
            cascade = CascadeType.ALL
    )
    @JoinColumn(
            name = "course_id",    ## create foreign key column named course_id
            referencedColumnName = "courseId"  ## refers to courseId in Course class

    )
    private Course course;
}

Następnie tworzymy dla obu klas repozytoria, a jeżeli chcemy przetestować działanie to tworzymy plik testowy dla CourseMaterial.

In [ ]:
@SpringBootTest
class CourseMaterialRepositoryTest {

    @Autowired
    private CourseMaterialRepository courseMaterialRepository;

    @Test
    public void SaveCourseMaterial() {

        Course course = Course.builder()
                .title("DSA")
                .credit(6)
                .build();

        CourseMaterial courseMaterial = CourseMaterial.builder()
                .url("www.google.com")
                .course(course)
                .build();

        courseMaterialRepository.save(courseMaterial);
    }
}

Niekiedy chcemy pobrać dane dla danej encji, bez pobierania danych powiązanych encji (np. chcemy pobrać nazwę firmy, ale bez pracujących tam pracowników itp.) Wówczas możemy zastosować dwa rodzaje "fetch-owania" - EAGERLY lub LAZY.

Różne relacje mają różne, domyślnie ustawione rodzaje fetchowania.

@OneToOne – FetchType.EAGER
@OneToMany – FetchType.LAZY
@ManyToOne – FetchType.EAGER
@ManyToMany – FetchType.LAZY

Jeżeli chcemy zastosować Fetch.LAZY dla relacji @OneToOne musimy dodać kod jak niżej

In [ ]:
@Entity
@Data
@AllArgsConstructor
@NoArgsConstructor
@Builder
@ToString(exclude = "course")    ## in tutorial this code was added, otherwise there was an error, but it worked for me without it
public class CourseMaterial {

    @Id
    @SequenceGenerator(
            name = "course_material_sequence",
            sequenceName = "course_material_sequence",
            allocationSize = 1
    )
    @GeneratedValue(
            strategy = GenerationType.SEQUENCE,
            generator = "course_material_sequence"
    )

    private Long courseMaterialId;
    private String url;

    @OneToOne(
            cascade = CascadeType.ALL,
            fetch = FetchType.LAZY,   ## here you set the fetch type
            optional = false ## it means that you can't add new course to DB without the related course material
    )
    @JoinColumn(
            name = "course_id",
            referencedColumnName = "courseId"

    )
    private Course course;
}


##### Relacja jedno lub dwukierunkowa

Na ten moment mamy ustawioną tylko relację jednokierunkową. W klasie encji CourseMaterial mamy ustawiony FOREIGN KEY dla atrybutu course. Aby zdefiniować relację dwukierunkową należy zmodyfikować klasę Course.

In [ ]:
@Entity
@Data
@AllArgsConstructor
@NoArgsConstructor
@Builder
public class Course {

    @Id
    @SequenceGenerator(
            name = "course_sequence",
            sequenceName = "course_sequence",
            allocationSize = 1
    )
    @GeneratedValue(
            strategy = GenerationType.SEQUENCE,
            generator = "course_sequence"
    )

    private Long courseId;
    private String title;
    private Integer credit;

    ## we have to add below code
    @OneToOne(
            mappedBy = "course",  ## refers to CourseMaterial's attribute which is set as a foreign key
            fetch = FetchType.LAZY
    )
    private CourseMaterial courseMaterial;
}

### Doświadczenie podczas tworzenia full-stackowej aplikacji.

Podczas tworzenia testowej bazy danych przy użyciu H2 stworzyłem schema.sql oraz data.sql. Miałem dwie tabele w relacji jeden do jednego - Jeden użytkownik może mieć tylko jeden wpis z tabeli Settings. Podczas tworzenia nowego użytkownika tworzymy nowy wpis w Setting

In [ ]:
public class AppUserService {

    @Autowired
    private AppUserRepository appUserRepository;

    @Autowired
    private SettingRepository settingRepository;

    @Autowired
    private SettingService settingService;

    public List<AppUser> getAllAppUsers() {
        return appUserRepository.findAll();
    }

    public AppUser addNewUser(AppUser newAppUser) { # pass new AppUser body from controller (@RequestBody...)
        Setting setting = new Setting();  # create new Setting entry
        setting.setAppUser(newAppUser);  # new setting entry requires to set newly created AppUser
        newAppUser.setSetting(setting);  # new user entry requires to set newly created Setting
        settingService.addSetting(setting);;    # saving new setting entry 
        appUserRepository.save(newAppUser);     # saving new user entry
        return newAppUser;
    }
}

Pojawił się również problem kiedy chciałem używałem metody GET aby podejrzeć wpisy Setting. Wyświetlały się ale bez "joinowanej" kolumny app_user_id. Okazało się, że muszę dodać kolumnę w encji Setting.

In [ ]:
@Entity
@Data
@Setter
@Getter
@Builder
@AllArgsConstructor
@NoArgsConstructor
public class Setting {

    @Id
    @GeneratedValue(
            strategy = GenerationType.IDENTITY
    )
    private Long settingId;
    @Column(name = "app_user_id", insertable = false, updatable = false) # this is the column I had to add. 
    private Long appUserId;
    @OneToOne(cascade = CascadeType.ALL)
    @JoinColumn(name = "app_user_id", referencedColumnName = "appUserId")
    @JsonIgnore
    @JsonProperty
    private AppUser appUser;
    @ManyToOne
    @JoinColumn(name = "theme_id")
    private Theme theme;
}

### result is:
{
    "settingId": 2,
    "appUserId": 3, #  it show the appUserId from joined column from AppUserEntity 
    "theme": {
        "themeId": 1,
        "name": "light"
}
###